In [1]:
import sys
import os

import pandas as pd
# import polars as pl

from matplotlib import pyplot as plt
%matplotlib inline  

sys.path.append('/Users/mihail/Documents/repos/analyzer')
print(sys.version)

from analyzer.stats import (
    calc_var_groups_stat,
    calc_iv_report, calc_iv_var, 
    calc_concentration_report,
    calc_stability_report
)

3.13.2 (main, Feb  4 2025, 14:51:09) [Clang 16.0.0 (clang-1600.0.26.6)]


# Подготовка
Загрузим датафрейм, на котором продемонстрируем функционал.

In [2]:
df = pd.read_csv('/Users/mihail/Documents/repos/analyzer/tests/Titanic-Dataset.csv')
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [3]:
df.shape

(891, 12)

In [4]:
df['Survived'].value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

# Функционал

## calc_var_groups_stat
Общая статистика по размеру популяций групп одной переменной.

In [5]:
?calc_var_groups_stat

Signature:
calc_var_groups_stat(
    df: ~DataFrame,
    var_name: str,
    map_values: Dict[~OldValue, ~NewValue] = None,
    binning: Union[analyzer.preprocessing.binning.params.BinningParams, bool] = True,
) -> ~DataFrame
Docstring:
Общая статистика по размеру популяций групп одной переменной.

Args:
    df:         Датафрейм, содержащий интересующую переменную.
    var_name:   Название интересующей переменной.
    map_values: Словарь для мэппинга значений переменной.
    binning:    Параметры для биннинга.

Returns:
    DataFrame:
        GROUP_NUMBER:   Номер группы
        GROUP:          Название группы
        TOTAL:          Размер группы, кол-во наблюдений
        POPULATION:     Размер группы, процент выборки
File:      ~/Documents/repos/analyzer/analyzer/stats/counts.py
Type:      function

In [6]:
calc_var_groups_stat(df, 'Survived')

/Users/mihail/Documents/repos/analyzer/analyzer/utils/framework_depends/columns/set.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = series_or_value
/Users/mihail/Documents/repos/analyzer/analyzer/stats/counts.py:58: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = df.groupby(var_name, as_index=False).size()


,GroupNumber,Group,Total,Population(%)
0,1,<= 0,549,61.62
1,2,> 0,342,38.38


## Information Value

### calc_iv_var
Расчет Information Value для одной переменной.

In [7]:
?calc_iv_var

Signature:
calc_iv_var(
    df: ~DataFrame,
    var_name: str,
    target_name: str,
    map_values: Dict[~OldValue, ~NewValue] = None,
    binning: Union[analyzer.preprocessing.binning.params.BinningParams, bool] = True,
    _validate_target: bool = True,
) -> ~DataFrame
Docstring:
Расчет Information Value для одной переменной.

Args:
    df:                 Исследуемый датафрейм
    var_name:           Название переменной
    target_name         Название таргета
    map_values:         Мэппинг значений переменной
    binning:            Параметры для биннинга или булев флаг (нужен или нет биннинг)
    _validate_target:   Скрытый параметр, валидация бинарности таргета

Returns:
    DataFrame:
        VARNAME:                имя переменной
        GROUP_NUMBER:           номер категории
        GROUP:                  значение категории
        COUNT:                  общий размер категории
        TARGET:                 количество таргетов в данной категории
        POPULATION:      

In [8]:
calc_iv_var(df, 'Age', 'Survived')

/Users/mihail/Documents/repos/analyzer/analyzer/utils/framework_depends/columns/set.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = series_or_value


,VarName,GroupNumber,Group,Count,Target,Population(%),TargetPopulation(%),TargetRate(%),GroupIV,TotalIV
0,Age,1,__MISSING__,177,52,19.87,15.20,29.378531,-3.054206,9.17652
1,Age,2,<= 18.5,139,70,15.60,20.47,50.359712,3.852415,9.17652
2,Age,3,(18.5; 26.5],180,60,20.20,17.54,33.333333,-0.948488,9.17652
3,Age,4,(26.5; 36.25],200,88,22.45,25.73,44.000000,1.237291,9.17652
4,Age,5,> 36.25,195,72,21.89,21.05,36.923077,-0.084119,9.17652


### calc_iv_report
Формирует отчет Information Value для заданных переменных.

In [9]:
?calc_iv_report

Signature:
calc_iv_report(
    df: ~DataFrame,
    target_name: str,
    analyze_vars: List[str] = None,
    ignore_vars: List[str] = None,
    map_values: Dict[~Variable, Dict[~OldValue, ~NewValue]] = None,
    binning: Union[Dict[str, Union[analyzer.preprocessing.binning.params.BinningParams, bool]], bool, analyzer.preprocessing.binning.params.BinningParams, List[str]] = True,
    sort_by_iv: bool = True,
    _tqdm: bool = True,
    _logging: bool = True,
) -> Tuple[~DataFrame, ~DataFrame]
Docstring:
Формирует отчет Information Value для заданных переменных.

Args:
    df:             Исследуемый датафрейм
    target_name:    Название таргета
    analyze_vars:   Список переменных для анализа, по умолчанию возьмутся все, кроме таргета и ignore_vars
    ignore_vars:    Список переменных игнорируемых во время анализа
    map_values:     Словарь для замены значений переменных (словарь, ключ = название переменной,
                    значение = словарь старое-новое значение)
    binning: 

In [11]:
total_iv, detail_iv = calc_iv_report(df, 'Survived')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Бинаризация переменных, 6/6 [00:00<00:00, 214.27it/s]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Расчет IV отчета, 11/11 [00:00<00:00, 346.72it/s]

2025-05-13 17:50:27,054 INFO 

    Var_Name - имя переменной,
    GroupNumber - номер категории,
    Group - значение категории,
    Count - общий размер категории,
    Target - количество таргетов в данной категории,
    Population(%) - относительный размер категории,
    TargetPopulation(%) - относительный размер таргета в данной категории,
    TargetRate(%) - Target Rate в данной категории,
    GroupIV - information value данной категории,
    TotalIV - information value всей переменной
    
    Information Value (IV):
    - Чем больше значение ПО МОДУЛЮ = тем больше отклонение target rate oт среднего по выборке (сильнее разделяющая способность)
    - IV > 0 = target rate выше среднего по выборке
    - IV < 0 = target rate ниже среднего по выборке




In [12]:
total_iv.head()

,VarName,TotalIV
0,Sex,134.168141
1,Ticket,79.470163
2,Fare,61.002363
3,Pclass,50.094974
4,Name,46.491635


In [13]:
detail_iv.head(5)

,VarName,GroupNumber,Group,Count,Target,Population(%),TargetPopulation(%),TargetRate(%),GroupIV,TotalIV
0,Sex,1,female,314,233,35.24,68.13,74.203822,81.656507,134.168141
1,Sex,2,male,577,109,64.76,31.87,18.890815,-52.511635,134.168141
2,Ticket,1,110152,3,3,0.34,0.88,100.000000,1.092539,79.470163
3,Ticket,2,110413,3,2,0.34,0.58,66.666667,0.469660,79.470163
4,Ticket,3,110465,2,0,0.22,0.00,0.000000,-0.015808,79.470163


## Concentrations
Отчет по концентрации таргета. Поиск аномальных сегментов.

In [10]:
?calc_concentration_report

Signature:
calc_concentration_report(
    df: ~DataFrame,
    target_name: str,
    combo_min: int = 1,
    combo_max: int = 1,
    analyze_vars: List[str] = None,
    ignore_vars: List[str] = None,
    map_values: Dict[~Variable, Dict[~OldValue, ~NewValue]] = None,
    binning: Union[Dict[str, Union[analyzer.preprocessing.binning.params.BinningParams, bool]], bool, analyzer.preprocessing.binning.params.BinningParams, List[str]] = True,
    pop_more: float = None,
    tr_less: float = None,
    tr_more: float = None,
    sort_by_iv: bool = False,
    _tqdm: bool = True,
    _validate_target: bool = True,
    _bin_by_target: bool = True,
    _logging: bool = True,
    _drop_single_vals: bool = True,
) -> ~DataFrame
Docstring:
Отчет по концентрации таргета. Поиск аномальных сегментов.

Args:
    df:                     Исследуемый датафрейм
    target_name:            Название таргета
    combo_min:              Минимальное количество переменных, участвующих в комбинировании
    combo_ma

In [15]:
report = calc_concentration_report(df, 'Survived', combo_max=4)
report.head()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Бинаризация переменных, 6/6 [00:00<00:00, 267.16it/s]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Комбинации из 1 переменных, 9/9 [00:00<00:00, 869.47it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Комбинации из 2 переменных, 36/36 [00:00<00:00, 1518.09it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Комбинации из 3 переменных, 84/84 [00:00<00:00, 1500.22it/s]
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Комбинации из 4 переменных, 126/126 [00:00<

2025-05-13 17:50:32,488 INFO 

    Var[i]_Name - имя i-ой переменной участвующей в сегменте,
    Var[i]_Value - значение i-ой переменной участвующей в сегменте,
    Count - количество наблюдений в данном сегменте,
    Population(%) - относительный размер сегмента,
    Target - количество таргетов в данном сегменте,
    TargetRate(%) - Target Rate в данном сегменте,
    TargetPopulation(%) - относительный размер таргета в данном сегменте,
    GroupIV - information value данного сегмента,

    ParentMin - родительский сегмент с минимальным target_rate,
    ParentMin TR(%) - минимальный target_rate родительского сегмента,
    ParentMax - родительский сегмент с максимальным target_rate,
    ParentMax TR(%) - максимальный target_rate родительского сегмента

    Information Value (IV):
    - Чем больше значение ПО МОДУЛЮ = тем больше отклонение target rate oт среднего по выборке (сильнее разделяющая способность)
    - IV > 0 = target rate выше среднего по выборке
    - IV < 0 = target rate н

,Var1_Name,Var1_Value,Var2_Name,Var2_Value,Var3_Name,Var3_Value,Var4_Name,Var4_Value,Count,Population(%),Target,TargetRate(%),TargetPopulation(%),GroupIV,ParentMin,ParentMin TR(%),ParentMax,ParentMax TR(%)
0,Age,> 36.25,None,None,None,None,None,None,195,21.89,72,36.923077,21.05,-0.084119,None,NaN,None,NaN
1,Age,__MISSING__,None,None,None,None,None,None,177,19.87,52,29.378531,15.20,-3.054206,None,NaN,None,NaN
2,Age,(18.5; 26.5],None,None,None,None,None,None,180,20.20,60,33.333333,17.54,-0.948488,None,NaN,None,NaN
3,Age,<= 18.5,None,None,None,None,None,None,139,15.60,70,50.359712,20.47,3.852415,None,NaN,None,NaN
4,Age,(26.5; 36.25],None,None,None,None,None,None,200,22.45,88,44.000000,25.73,1.237291,None,NaN,None,NaN


In [17]:
report = calc_concentration_report(df, 'Survived', combo_max=4, sort_by_iv=True)
report.head()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Бинаризация переменных, 6/6 [00:00<00:00, 234.10it/s]
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Комбинации из 1 переменных, 9/9 [00:00<00:00, 1559.80it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Комбинации из 2 переменных, 36/36 [00:00<00:00, 1500.00it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Комбинации из 3 переменных, 84/84 [00:00<00:00, 1524.78it/s]
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Комбинации из 4 переменных, 126/126 [00:00<

2025-05-13 17:50:45,591 INFO 

    Var[i]_Name - имя i-ой переменной участвующей в сегменте,
    Var[i]_Value - значение i-ой переменной участвующей в сегменте,
    Count - количество наблюдений в данном сегменте,
    Population(%) - относительный размер сегмента,
    Target - количество таргетов в данном сегменте,
    TargetRate(%) - Target Rate в данном сегменте,
    TargetPopulation(%) - относительный размер таргета в данном сегменте,
    GroupIV - information value данного сегмента,

    ParentMin - родительский сегмент с минимальным target_rate,
    ParentMin TR(%) - минимальный target_rate родительского сегмента,
    ParentMax - родительский сегмент с максимальным target_rate,
    ParentMax TR(%) - максимальный target_rate родительского сегмента

    Information Value (IV):
    - Чем больше значение ПО МОДУЛЮ = тем больше отклонение target rate oт среднего по выборке (сильнее разделяющая способность)
    - IV > 0 = target rate выше среднего по выборке
    - IV < 0 = target rate н

,Var1_Name,Var1_Value,Var2_Name,Var2_Value,Var3_Name,Var3_Value,Var4_Name,Var4_Value,Count,Population(%),Target,TargetRate(%),TargetPopulation(%),GroupIV,ParentMin,ParentMin TR(%),ParentMax,ParentMax TR(%)
0,Pclass,<= 1,Sex,female,None,None,None,None,94,10.55,91,96.808511,26.61,101.263797,- Sex,62.962963,- Pclass,74.203822
1,Fare,> 52.277099609375,Pclass,<= 1,Sex,female,None,None,78,8.75,76,97.435897,22.22,89.855166,- Sex,75.781250,- Fare,96.808511
2,Parch,<= 0,Pclass,<= 1,Sex,female,None,None,64,7.18,63,98.437500,18.42,84.198482,- Sex,60.736196,- Parch,96.808511
3,Sex,female,None,None,None,None,None,None,314,35.24,233,74.203822,68.13,81.656507,None,NaN,None,NaN
4,Fare,> 52.277099609375,Sex,female,None,None,None,None,83,9.32,78,93.975904,22.81,70.518223,- Sex,69.798658,- Fare,74.203822


## Stability 

In [19]:
?calc_stability_report

Signature:
calc_stability_report(
    df: ~DataFrame,
    split_var_name: str,
    combo_min: int = 1,
    combo_max: int = 1,
    analyze_vars: List[str] = None,
    ignore_vars: List[str] = None,
    split_var_value: Any = None,
    target_name: str = None,
    map_values: Dict[~Variable, Dict[~OldValue, ~NewValue]] = None,
    binning: Union[Dict[str, Union[analyzer.preprocessing.binning.params.BinningParams, bool]], bool, analyzer.preprocessing.binning.params.BinningParams, List[str]] = True,
    min_part_or_cnt: Union[float, int] = None,
    min_abs_popstab: float = None,
    min_abs_tgstab: float = None,
) -> ~DataFrame
Docstring:
Отчет по стабильности переменных

Args:
    df:                         Исследуемый датафрейм
    split_var_name:             Переменная которая будет разбивать выборку на подвыборки
    combo_min:                  Минимальное количество переменных, участвующих в комбинировании
    combo_max:                  Максимальное количество переменных, участвую

In [21]:
stability_report = calc_stability_report(
        df, split_var_name='Pclass', combo_max=2,
        target_name='Survived', split_var_value=1,
        min_part_or_cnt=0.1
    )
stability_report.head()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Бинаризация переменных, 5/5 [00:00<00:00, 242.06it/s]

2025-05-13 17:51:00,884 INFO 

    SplitColumn - поле, которое разбивает данные на подвыборки для сверки их характеристик между собой,
    Var[i]_Name - имя i-ой переменной участвующей в сегменте, 
    Var[i]_Value - значение i-ой переменной участвующей в сегменте, 
    CountMain - количество наблюдений в данном сегменте базовой выборки (относительно которой будем сверять),
    CountSecondary - количество наблюдений в данном сегменте в сверяемой выборке (определяется по SplitColumn),
    PopulationMain(%) - относительный размер сегмента в базовой выборке, 
    PopulationSecondary(%) - относительный размер сегмента в сверяемой выборке, 
    TargetMain - количество таргетов в данном сегменте базовой выборки, 
    TargetSecondary - количество таргетов в данном сегменте в сверяемой выборке
    TargetRateMain(%) - Target Rate в данному сегменте базовой выборки, 
    TargetRateSecondary(%) - Target Rate в данному сегменте в сверяемой выборке
    TargetPop-nMain(%) - относительный размер тарг

,Pclass,Var1_Name,Var1_Value,Var2_Name,Var2_Value,CountMain,CountSecondary,PopulationMain(%),PopulationSecondary(%),TargetMain,TargetSecondary,TargetRateMain(%),TargetRateSecondary(%),TargetPop-nMain(%),TargetPop-nSec-ry(%),Pop-nStability,TargetStability
0,1,Sex,female,None,None,94,94,43.518519,43.518519,91,91,96.808511,96.808511,66.911765,66.911765,0.000000,0.000000
1,1,Sex,male,None,None,122,122,56.481481,56.481481,45,45,36.885246,36.885246,33.088235,33.088235,0.000000,0.000000
2,2,Sex,female,None,None,94,76,43.518519,41.304348,91,70,96.808511,92.105263,66.911765,80.459770,-0.115621,2.498015
3,2,Sex,male,None,None,122,108,56.481481,58.695652,45,17,36.885246,15.740741,33.088235,19.540230,0.085141,-7.135767
4,3,Sex,female,None,None,94,144,43.518519,29.327902,91,72,96.808511,50.000000,66.911765,60.504202,-5.600287,-0.644998
